In [ ]:
import numpy as np
import torch
from ffnet2 import FFNet2
from mfn import GaborNetwork
import matplotlib.pyplot as plt

# Function to load and transform data
def load_and_transform_data(file_path):
    original_data = np.load(file_path).astype('int16')
    transformed_data = original_data.astype(float)
    valid_mask = transformed_data > 64
    transformed_data[valid_mask] = np.log(transformed_data[valid_mask] - 64) / 6
    return original_data, transformed_data

def plot_data(arr1, arr2):
    # Create the figure
    fig = plt.figure(figsize=(24, 8))  # Adjusted width to accommodate two plots

    # Create first subplot for the first array
    ax1 = fig.add_subplot(121, projection='3d')  # 121 means 1 row, 2 columns, first subplot

    # Create meshgrid for plotting
    x1, y1 = np.meshgrid(np.arange(arr1.shape[1]), np.arange(arr1.shape[0]))

    # Plot each layer for the first array
    for i in range(arr1.shape[2]):
        z1 = i * np.ones_like(x1)
        c1 = arr1[:,:,i]
        ax1.plot_surface(x1, y1, z1, facecolors=plt.cm.Reds(np.abs(c1)), 
                        rstride=1, cstride=1, alpha=0.3, shade=False)

    # Set labels and title for the first subplot
    ax1.set_xlabel('azimuthal')
    ax1.set_ylabel('z')
    ax1.set_zlabel('layer')
    ax1.set_title('3D Heatmap of sin(x) Array - Dataset 1')

    # Set axis limits for the first subplot
    ax1.set_xlim(0, arr1.shape[1])
    ax1.set_ylim(0, arr1.shape[0])
    ax1.set_zlim(0, arr1.shape[2])

    # Adjust the view angle for the first subplot
    ax1.view_init(elev=20, azim=-45)

    # Create second subplot for the second array
    ax2 = fig.add_subplot(122, projection='3d')  # 122 means 1 row, 2 columns, second subplot

    # Create meshgrid for plotting
    x2, y2 = np.meshgrid(np.arange(arr2.shape[1]), np.arange(arr2.shape[0]))

    # Plot each layer for the second array
    for i in range(arr2.shape[2]):
        z2 = i * np.ones_like(x2)
        c2 = arr2[:,:,i]
        ax2.plot_surface(x2, y2, z2, facecolors=plt.cm.Reds(np.abs(c2)), 
                        rstride=1, cstride=1, alpha=0.3, shade=False)

    # Set labels and title for the second subplot
    ax2.set_xlabel('azimuthal')
    ax2.set_ylabel('z')
    ax2.set_zlabel('layer')
    ax2.set_title('3D Heatmap of sin(x) Array - Dataset 2')

    # Set axis limits for the second subplot
    ax2.set_xlim(0, arr2.shape[1])
    ax2.set_ylim(0, arr2.shape[0])
    ax2.set_zlim(0, arr2.shape[2])

    # Adjust the view angle for the second subplot
    ax2.view_init(elev=20, azim=-45)

    plt.show()

def test_single_model(ModelClass, hyperparameters, original_data, transformed_data):
    model = ModelClass(hyperparameters)
    
    print(f"\nEvaluating {ModelClass.__name__}")
    print("Hyperparameters:")
    for key, value in hyperparameters.items():
        print(f"  {key}: {value}")
    
    # Train the model on transformed data
    mse_transformed = model.train(transformed_data)

    # Test the model on the same data
    array_loader = model.create_loader(transformed_data)
    grid, array = next(iter(array_loader))
    grid, array = grid.squeeze().to(model.device), array.squeeze().to(model.device)
    coords, values = grid.reshape(-1, 3), array.reshape(-1, 1)
    prediction_transformed = model.predict(coords)
    
    # Reshape prediction to original shape
    original_shape = array.cpu().numpy().shape
    predicted_array_transformed = prediction_transformed.reshape(original_shape).cpu().numpy()
    
    # Inverse transform the predictions (adjusted for mostly 0s)
    predicted_array = np.zeros_like(original_data, dtype=float)
    non_zero_mask = predicted_array_transformed > 0
    predicted_array[non_zero_mask] = np.exp(predicted_array_transformed[non_zero_mask] * 6) + 64
    
    # Apply threshold to set small values to zero
    threshold = 65  # Adjust this value as needed
    predicted_array[predicted_array < threshold] = 0

    # Calculate MSE on original (non-log) scale
    mse_original = np.mean((original_data - predicted_array)**2)
    
    # Plot the original and predicted data
    plot_data(original_data, predicted_array)
    
    # Compute compression ratio
    compression_ratio = model.get_compression_ratio(original_data.size)
    
    print(f"MSE (original scale): {mse_original}")
    print(f"MSE (transformed scale): {mse_transformed}")
    print(f"Compression Ratio: {compression_ratio}")
    
    # Print some statistics about the predicted array
    non_zero_count = np.count_nonzero(predicted_array)
    total_elements = predicted_array.size
    print(f"Non-zero elements: {non_zero_count} out of {total_elements} ({non_zero_count/total_elements*100:.2f}%)")
    
    return {
        'model': ModelClass.__name__,
        'original_mse': mse_original,
        'transformed_mse': mse_transformed,
        'compression_ratio': compression_ratio,
        'predicted_array': predicted_array,
        **hyperparameters
    }

# Load and transform a single data sample
data_path = "outer/12-2_9-1/AuAu200_170kHz_10C_Iter2_3165.xml_TPCMLDataInterface_2.npy"  # Replace with actual path
original_data = np.load(data_path).astype('int16')
original_data, transformed_data = load_and_transform_data(data_path)

# Define hyperparameters for FFNet
ffnet_hyperparameters = {
    'lr': 1e-3,
    'in_features': 3,
    'hidden_features': 64,
    'hidden_layers': 3,
    'out_features': 1,
    'input_scale': 256.0,
    'weight_scale': 1.0,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Test FFNet
ffnet_result = test_single_model(FFNet2, ffnet_hyperparameters, original_data, transformed_data)

# Define hyperparameters for GaborNetwork
gabor_hyperparameters = {
    'lr': 1e-3,
    'in_features': 3,
    'hidden_features': 64,
    'hidden_layers': 3,
    'out_features': 1,
    'input_scale': 256.0,
    'weight_scale': 1.0,
    'alpha': 6.0,
    'beta': 1.0,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Test GaborNetwork
gabor_result = test_single_model(GaborNetwork, gabor_hyperparameters, original_data, transformed_data)

# Print results
print("\nFFNet Results:")
print(ffnet_result)
print("\nGaborNetwork Results:")
print(gabor_result)

# Access predicted arrays
ffnet_predicted_array = ffnet_result['predicted_array']
gabor_predicted_array = gabor_result['predicted_array']



Evaluating FFNet2
Hyperparameters:
  lr: 0.001
  in_features: 3
  hidden_features: 64
  hidden_layers: 3
  out_features: 1
  input_scale: 256.0
  weight_scale: 1.0
  device: cuda
Step: 100, Test MSE: 0.015745
Step: 200, Test MSE: 0.012759
Step: 300, Test MSE: 0.011791
Step: 400, Test MSE: 0.011360
Step: 500, Test MSE: 0.011103
Step: 600, Test MSE: 0.010915
Step: 700, Test MSE: 0.010780
Step: 800, Test MSE: 0.010678
Step: 900, Test MSE: 0.010591
Step: 1000, Test MSE: 0.010517
